<a href="https://colab.research.google.com/github/hamuim/Patern-idea-TensorFlow/blob/main/HWSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/hamuim/Patern-idea-TensorFlow

Cloning into 'Patern-idea-TensorFlow'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 6.75 KiB | 6.75 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
import sys
sys.path.append("Patern-idea-TensorFlow")

In [ ]:
import tensorflow as tf
tf.random.set_seed(42)

In [ ]:
try:
    tpu = None
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
import matplotlib.pyplot as plt
import rn10
import ut
import time

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
BATCH_SIZE = 128 * strategy.num_replicas_in_sync
AUTO = tf.data.AUTOTUNE

def scale(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.cast(label, tf.int32)
    return image, label

def augment(image,label):
    image = tf.image.resize_with_crop_or_pad(image, 40, 40)
    image = tf.image.random_crop(image, size=[32, 32, 3])
    image = tf.image.random_brightness(image, max_delta=0.5)
    image = tf.clip_by_value(image, 0., 1.)

    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = (
    train_ds
    .shuffle(1024)
    .map(scale, num_parallel_calls=AUTO)
    .map(augment, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = (
    test_ds
    .map(scale, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(False)

class HWSHModel(tf.keras.Model):
    def __init__(self, resnet_model, rho=0.05):
        super(HWSHModel, self).__init__()
        self.resnet_model = resnet_model
        self.rho = rho

    def train_step(self, data):
        (images, labels) = data
        e_ws = []
        with tf.GradientTape() as tape:
            predictions = self.resnet_model(images)
            loss = self.compiled_loss(labels, predictions)
        trainable_params = self.resnet_model.trainable_variables
        gradients = tape.gradient(loss, trainable_params)
        grad_norm = self._grad_norm(gradients)
        scale = self.rho / (grad_norm + 1e-12)

        for (grad, param) in zip(gradients, trainable_params):
            e_w = grad * scale
            param.assign_add(e_w)
            e_ws.append(e_w)

        with tf.GradientTape() as tape:
            predictions = self.resnet_model(images)
            loss = self.compiled_loss(labels, predictions)

        sam_gradients = tape.gradient(loss, trainable_params)
        for (param, e_w) in zip(trainable_params, e_ws):
            param.assign_sub(e_w)

        self.optimizer.apply_gradients(
            zip(sam_gradients, trainable_params))

        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        (images, labels) = data
        predictions = self.resnet_model(images, training=False)
        loss = self.compiled_loss(labels, predictions)
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def _grad_norm(self, gradients):
        norm = tf.norm(
            tf.stack([
                tf.norm(grad) for grad in gradients if grad is not None
            ])
        )
        return norm

In [ ]:
train_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5,
        patience=3, verbose=1
    )
]

In [ ]:
with strategy.scope():
    model = tf.keras.models.experimental.SharpnessAwareMinimization(ut.get_training_model1(), rho=0.05, num_batch_splits=None, name=None)
model.compile(
    optimizer="SGD", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
start = time.time()
history = model.fit(train_ds,
                   validation_data=test_ds,
                   callbacks=train_callbacks,
                   epochs=100)

Epoch 1/100
391/391 [==============================] - 18s 45ms/step - loss: 2.1092 - accuracy: 0.2238 - val_loss: 1.9752 - val_accuracy: 0.3211 - lr: 0.0010
Epoch 2/100
391/391 [==============================] - 18s 45ms/step - loss: 1.8636 - accuracy: 0.3178 - val_loss: 1.8116 - val_accuracy: 0.3919 - lr: 0.0010
Epoch 3/100
391/391 [==============================] - 17s 42ms/step - loss: 1.7467 - accuracy: 0.3661 - val_loss: 1.7426 - val_accuracy: 0.4236 - lr: 0.0010
Epoch 4/100
391/391 [==============================] - 16s 41ms/step - loss: 1.6689 - accuracy: 0.4034 - val_loss: 1.6804 - val_accuracy: 0.4450 - lr: 0.0010
Epoch 5/100
391/391 [==============================] - 16s 42ms/step - loss: 1.5907 - accuracy: 0.4378 - val_loss: 1.6279 - val_accuracy: 0.4737 - lr: 0.0010
Epoch 6/100
391/391 [==============================] - 16s 41ms/step - loss: 1.5242 - accuracy: 0.4613 - val_loss: 1.5361 - val_accuracy: 0.5133 - lr: 0.0010
Epoch 7/100
391/391 [==============================]

KeyboardInterrupt: ignored

In [ ]:
ut.plot_history(history)

In [ ]:
with strategy.scope():
    model = ut.get_training_model1()

model.compile(
    optimizer="SGD", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

start = time.time()
history = model.fit(train_ds,
                   validation_data=test_ds,
                   callbacks=train_callbacks,
                   epochs=5)
print(f"Total training time: {(time.time() - start)/60.} minutes")

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/5
391/391 [==============================] - 590s 1s/step - loss: 0.8022 - accuracy: 0.7281 - val_loss: 2.9041 - val_accuracy: 0.1008 - lr: 0.0100
Epoch 2/5
112/391 [=======>......................] - ETA: 6:06 - loss: 0.3804 - accuracy: 0.8695

In [ ]:
ut.plot_history(history)